![](https://api.brandy.run/core/core-logo-wide)

## Partial and currying

El objetivo del paradigma de programacion funcional es tener, no una unica funcion de grandes proporciones y complejidad, sino muchas funciones, extremadamente simples. Estas funciones deben resolver partes de un problema más global, y los parametros que deben recibir deben de ser los minimos posibles, es decir, deben apuntar a tener una **Aridad unitaria**.

Las ventajas de tener multiples funciones, frete a una sola función, es que permite tener "modularidad", ya que al tener funciones que resuelven partes del problema, se pueden aplicar a multiples problemas unicamente teniendo que cambiar/añadir funciones para que actuen correctamente en el nuevo contexto.

Esta situacion, para las funciones unicas de gran tamaño, supone un problema, ya que habría que buscar por todo su codigo, aquella parte de él que ha dejado de funcionar y "arreglarla" para el nuevo contexto, lo que haría que dejase de funcionar para el contexto anterior.

A continuacion, vamos a simular una conexion a una BBDD para obtener algun tipo de información.

In [1]:
def connection(bbdd):
    print(f"Conectandose a la bbdd {bbdd} ....\n...")
    return bbdd

def connected(table, bbdd):
    print(f"Se ha conectado a la coleccion {table} de {bbdd}.\n...")
    return True

def read(conexion):
    data = {"Name":"Pepe", "Age":40, "Place":"Madrid"}
    print(f"Se han encontrado los siguientes datos en la tabla:")
    for key,value in data.items():
        print(f"   {key}:{value}")
    print("")
    return data

def data(data, feature):
    print(f"Buscando por {feature} ...")
    return data[feature]

Para poder conectarse a la BBDD y obtener la informacion debemos segir el siguiente flujo de ejecucion

In [2]:
bbdd = connection("Bootcamp")
conexion = connected("Alumnos", bbdd)
data_bbdd = read(conexion)
print(data(data_bbdd, "Name"))

Conectandose a la bbdd Bootcamp ....
...
Se ha conectado a la coleccion Alumnos de Bootcamp.
...
Se han encontrado los siguientes datos en la tabla:
   Name:Pepe
   Age:40
   Place:Madrid

Buscando por Name ...
Pepe


Si utilizamos una funcion que componga todas estas funciones, podemos separar la intruduccion de las funciones y la introduccion de los datos:

In [3]:
def connection_process(connection, connected, read, data):
    return lambda table, bbdd, feature: data(read(connected(table, connection(bbdd))),feature)

In [4]:
connection_process(connection, connected, read, data)("Alumnos","Core","Name")

Conectandose a la bbdd Core ....
...
Se ha conectado a la coleccion Alumnos de Core.
...
Se han encontrado los siguientes datos en la tabla:
   Name:Pepe
   Age:40
   Place:Madrid

Buscando por Name ...


'Pepe'

De esta forma, hemos separado el proceso en dos. Por un lado, se establece la conexion a la bbdd. Por otro lado, se le indica de donde se quieren obtener los metodos. Debido a como funciona `python`, podemos almacenar parte del proceso en una variable, lo que nos ahorraria tener que estar llamando en todo momento a la totalidad de la funcion.

In [5]:
conexion = connection_process(connection, connected, read, data)
conexion("Alumnos","Core","Name")

Conectandose a la bbdd Core ....
...
Se ha conectado a la coleccion Alumnos de Core.
...
Se han encontrado los siguientes datos en la tabla:
   Name:Pepe
   Age:40
   Place:Madrid

Buscando por Name ...


'Pepe'

Se ha acortado el codigo, pero ahora es algo mas complejo de leer. Se puede saber que es una composicion de una conexion, una confirmacion, una lectura de los datos de la bbd y por ultimo, el dato que queremos sacar

Con Currying, podemos simplificar esto algo más

In [6]:
def full_connection(connection):
    def connect_f(connect):
        def table_to_connect(table):
            def bbdd_to_connect(bbdd):
                c = connection(bbdd)
                established_connection = connect(table, c)
                return established_connection
            return bbdd_to_connect
        return table_to_connect
    return connect_f

In [7]:
full_connection(connection)(connected)("Alumnos")("Bootcamp")
conexion = full_connection(connection)(connected)

Conectandose a la bbdd Bootcamp ....
...
Se ha conectado a la coleccion Alumnos de Bootcamp.
...


In [8]:
conexion_stablished = conexion("Alumno")("Bootcamp")

Conectandose a la bbdd Bootcamp ....
...
Se ha conectado a la coleccion Alumno de Bootcamp.
...


In [9]:
def curry_get_data(read):
    def get_data_of(connexion):
        def get_data(data):
            def by_feature(feature):
                lectura = read(connexion)
                return data(lectura,feature)
            return by_feature
        return get_data
    return get_data_of

In [10]:
 curry_get_data(read)(conexion_stablished)(data)("Name")

Se han encontrado los siguientes datos en la tabla:
   Name:Pepe
   Age:40
   Place:Madrid

Buscando por Name ...


'Pepe'

Aqui, se está haciendo bastante dificil, ya que tengo funciones con una aridad mayor que 1, y estoy necesitando pasarle hasta 4 parametros, lo que implica hacer hasta 4 definiciones de funcion dentro de la composición. La mayor ventaja del currying es que me permite declarar atajos para una parte de la composicion de la funcion. Estos atajos luego se pueden modificar más tarde en el codigo.

La verdadera complicacion del currying (o en donde se encuentra su mayor dificultad), es en la definicion del return "más" profundo. Esto se debe a que debemos empezar a devolver (incluir en el return) las llamadas de las funciones en orden inverso, es decir, empezando desde el final hasta el principio.

En este ultimo ejemplo, la funcion `data` es la que devuelve los datos que queremos obtener, por lo tanto debe ser la funcion que se encuentre más a la izquierda. A partir de ahi, solo hayq eu ver que parametro neesita recibir y quien se lo puede proporcionar. Pasa a ser simplemente un "juego" de ver que funcion proporciona dicho valor.

El currying nos puede ser de gran utilidad para la composicion de funciones, pero el proceso del currying puede ser algo confuso de hacer y relativamente facil de equivocarte en el orden de las funciones. Es por eso que, por suerte, ya existen funciones que nos permiten hacer es currying mucho más rapido y sencillo.

Estas funciones las importamos de la libreria `functools`. La funcion que usaremos de esta libreria será `partials`

In [11]:
from functools import partial

In [12]:
 connection_process(connection, connected, read, data)("Alumnos","Core","Name")

Conectandose a la bbdd Core ....
...
Se ha conectado a la coleccion Alumnos de Core.
...
Se han encontrado los siguientes datos en la tabla:
   Name:Pepe
   Age:40
   Place:Madrid

Buscando por Name ...


'Pepe'

Con partial, podemos dar un valor determinado a los parametros de una funcion. A continuacion, podemos ver una llamada a una funcion la cual necesita 4 y 3 parametros:
```python
connection_process(connection, connected, read, data)("Alumnos","Core","Name")
```

Con los partial, podemos predefinir los parametros `connection` y `connected`

In [13]:
conexion = partial(connection_process, connection, connected)

In [17]:
conexion_and_data = conexion(read, data)

In [18]:
conexion_and_data("Alumnos","Core","Name")

Conectandose a la bbdd Core ....
...
Se ha conectado a la coleccion Alumnos de Core.
...
Se han encontrado los siguientes datos en la tabla:
   Name:Pepe
   Age:40
   Place:Madrid

Buscando por Name ...


'Pepe'

Sin el partial, no se podría hacer esto ya que nos daría error porque la funciones estaría esperando los 4 parametros y solo le estaríamos definiendo 2.

Ni siquiera, pasandole los parametros como posicionales, podriamos hacerlo.

In [20]:
c2 = connection_process(connection=connection, connected=connected)

TypeError: connection_process() missing 2 required positional arguments: 'read' and 'data'

La forma de hacerlo, sería con una funcion `lambda` de la siguiente forma.

In [ ]:
c2 =  lambda read, data, feature: connection_process(connection, connected, read, data, feature)

In [ ]:
c2(read, data, "Name")